In [1]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
from glob import glob
from PIL import Image
import pickle
import cv2
from sklearn.utils import shuffle

# print(tf.__version__) 

from IPython.display import clear_output, display
import os.path
from os import path
import shutil
import PIL
from pathlib import Path
from matplotlib import pyplot as plt

from IPython.display import clear_output
import random
import re

In [12]:
from pathlib import Path    

In [2]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [6]:
#
# le labels em ingles
#
pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
with open(pgnfile) as f:
    en_lines = [line.rstrip() for line in f]
    
en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
en_lines= [ln[8:] for ln in en_lines]
en_lines= [ln.split()[0:16] for ln in en_lines]

In [7]:
en_lines[0]

['d4',
 'f5',
 'Nf3',
 'g6',
 'g3',
 'Bg7',
 'Bg2',
 'Nf6',
 'b4',
 'c6',
 'c4',
 'd5',
 'Qb3',
 'Be6',
 'c5',
 'Ne4']

In [10]:
#
# le cada um dos labels do dataset e ve se está igual
#
d= "train"
labelfiles= glob( os.path.join( 
    "C:/mestrado/datasets/unified/final-32K-8lines", 
    d, "labels", "syn*.pgn"))
                                    

In [11]:
len( labelfiles)

20217

In [18]:
Path( labelfiles[0]).name

'syn_00049.pgn'

In [21]:
def lnOf( f):
    return int( Path( f).name[4:9])

lnOf( labelfiles[0])

49

In [24]:
labels= []
for f in labelfiles:
    labels.append( read_label( f))

In [25]:
labels[0]

'e4 c5 Nc3 Nc6 Nf3 d6 d4 cxd4 Nxd4 Nf6 Be2 e5 Nb3 Be6 Bg5 a5'

In [27]:
en_lines[lnOf( labelfiles[0])-1]

['e4',
 'c5',
 'Nc3',
 'Nc6',
 'Nf3',
 'd6',
 'd4',
 'cxd4',
 'Nxd4',
 'Nf6',
 'Be2',
 'e5',
 'Nb3',
 'Be6',
 'Bg5',
 'a5']

In [29]:
pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
with open(pgnfile) as f:
    en_lines = [line.rstrip() for line in f]
    
en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
en_lines= [ln[8:] for ln in en_lines]
#en_lines= [ln.split() for ln in en_lines]

In [30]:
en_lines[0]

'd4 f5 Nf3 g6 g3 Bg7 Bg2 Nf6 b4 c6 c4 d5 Qb3 Be6 c5 Ne4 O-O Nd7 Bf4 O-O Nbd2 Bf7 Qe3 Qe8 Bh6 Ndf6 Ne5 Nxd2 Qxd2 Nd7 Bxg7 Kxg7 Qf4 Be6 a4 Nxe5 Qxe5+ Rf6 f4 Kg8 Rfb1 Bc8 Qc7 Qd7 Qxd7 Bxd7 b5 Rff8 e3 e6 Bf1 Kf7 a5 Rfb8 Rb2 Ke7 Rab1 Rc8 bxc6 bxc6 Ba6 Rc7 Rb8 Rxb8 Rxb8 Be8 Bb7 Bd7 a6 Be8 Ra8 Bd7 Kf2 Kf6 Rh8 Kg7 Ra8 Kf6 Ke2 h6 Kd2 g5 Kc3 gxf4 exf4'

In [31]:
all_lines= "\n".join( en_lines)

In [32]:
flabels= "C:/mestrado/datasets/unified/parts/vocabulary/all-30k-eng-labels.txt"
write_label( flabels, all_lines)